In [9]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [10]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [11]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [13]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Chance the Rapper']

In [14]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468 entries, 316 to 5308
Data columns (total 5 columns):
song_owner      468 non-null object
song_title      468 non-null object
song_element    468 non-null object
verse           468 non-null object
artist          468 non-null object
dtypes: object(5)
memory usage: 21.9+ KB


In [15]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [16]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],"Eddy Scissorhands is workin' on my linin', Get...",Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],"You are the prototype, I love you like you lov...",Chance the Rapper,"[you, are, the, prototype, i, love, you, like,..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],"Chance, acid rapper, soccer, hacky sacker, Coc...",Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],"This shit my favorite song, you just don't kno...",Chance the Rapper,"[this, shit, my, favorite, song, you, just, do..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],Young Rascal Flatts - young ass kid ass could ...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as..."


In [17]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [18]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],"Eddy Scissorhands is workin' on my linin', Get...",Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],"You are the prototype, I love you like you lov...",Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],"Chance, acid rapper, soccer, hacky sacker, Coc...",Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],"This shit my favorite song, you just don't kno...",Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],Young Rascal Flatts - young ass kid ass could ...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [19]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [20]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

316    eddy scissorhands is workin on my linin gettin...
793    you are the prototype i love you like you love...
824    chance acid rapper soccer hacky sacker cocky k...
825    this shit my favorite song you just dont know ...
826    young rascal flatts young ass kid ass could ra...
Name: verse, dtype: object

In [21]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],eddy scissorhands is workin on my linin gettin...,Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],you are the prototype i love you like you love...,Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],chance acid rapper soccer hacky sacker cocky k...,Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],this shit my favorite song you just dont know ...,Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],young rascal flatts young ass kid ass could ra...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [22]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [23]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],eddy scissorhands is workin on my linin gettin...,Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini...","[eddy, scissorhands, is, workin, ', on, my, li..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],you are the prototype i love you like you love...,Chance the Rapper,"[you, are, the, prototype, i, love, you, like,...","[you, are, the, prototype, ,, i, love, you, li..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],chance acid rapper soccer hacky sacker cocky k...,Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ...","[chance, ,, acid, rapper, ,, soccer, ,, hacky,..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],this shit my favorite song you just dont know ...,Chance the Rapper,"[this, shit, my, favorite, song, you, just, do...","[this, shit, my, favorite, song, ,, you, just,..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],young rascal flatts young ass kid ass could ra...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as...","[young, rascal, flatts, -, young, ass, kid, as..."


In [24]:
get_words(model_df)

['eddy',
 'scissorhands',
 'is',
 'workin',
 'on',
 'my',
 'linin',
 'gettin',
 'spiffy',
 'for',
 'these',
 'bitches',
 'i',
 'be',
 'eyeing',
 'fredward',
 'mercury',
 'was',
 'playin',
 'while',
 'im',
 'ridin',
 'down',
 'the',
 'ryan',
 'in',
 'a',
 'scion',
 'and',
 'im',
 'flyin',
 'but',
 'its',
 'cool',
 'i',
 'got',
 'insurance',
 'on',
 'the',
 'run',
 'because',
 'i',
 'got',
 'warrants',
 'but',
 'its',
 'cool',
 'i',
 'got',
 'endurance',
 'in',
 'school',
 'i',
 'got',
 'suspended',
 'but',
 'in',
 'turn',
 'i',
 'got',
 'some',
 'earnings',
 'couldve',
 'spent',
 'it',
 'all',
 'on',
 'thursday',
 'but',
 'i',
 'saved',
 'it',
 'for',
 'my',
 'parents',
 'but',
 'i',
 'spent',
 'a',
 'bit',
 'on',
 'friday',
 'in',
 'the',
 'night',
 'i',
 'hang',
 'with',
 'donald',
 'im',
 'tired',
 'of',
 'mcdonalds',
 'want',
 'a',
 'chick',
 'that',
 'cook',
 'chicano',
 'that',
 'cant',
 'speak',
 'a',
 'lick',
 'of',
 'english',
 'and',
 'pronounce',
 'my',
 'name',
 'like',
 'ch

In [25]:
verses = list(model_df.verse.values)

In [26]:
verses[:10]

['eddy scissorhands is workin on my linin gettin spiffy for these bitches i be eyeing fredward mercury was playin while im ridin down the ryan in a scion and im flyin but its cool i got insurance on the run because i got warrants but its cool i got endurance in school i got suspended but in turn i got some earnings couldve spent it all on thursday but i saved it for my parents but i spent a bit on friday in the night i hang with donald im tired of mcdonalds want a chick that cook chicano that cant speak a lick of english and pronounce my name like chano she say something something spanish i look back like bitch i know tonight imma make decisions for life take some lucys boomers dark liq and light some might drive thats that shit right there that i dont like there that shit just had to say shouts to the niggas that passed away and shots for the niggas thats here right now and shots at the niggas that got away so sad to say but its all good niggas gon celebrate and wait for the day and w

In [27]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [28]:
count, following = make_word_dictionary(verses)

In [29]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [30]:
verses_bigrams = create_bigrams(verses)

In [31]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [32]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

drugs no goofie i aint goin to hell holy grail that all these things all all all i can be but that 10day done in one night honda from a hammy to the


red dolls and green rangers things ain t on shit if you ask me aint no where brighter aint no goofie i aint even write this down i burned too many brain cells


friends labels told me go on and rock it i did my worst i did a ton of drugs is enormous recently a huge supply of heroin was taken it was just a


brother foot work your play sister juke the rink you had to take the campers way downtown you gon see us laid up put that on my name and started missing my chance


cap then act up john cena chancellor bennett referred to as word chemist the bourbon and slurred sentences is turning up burning bridges cigarette stained smile all covered in sin my big homie


streets we gon shake this shit up keep on dancin babe all i got the bill thats cool though now you know who i am his stand in i mean hit me better


club is six blunt rotation musi

brother foot on the wall i might give and go in stores and take a lot only the goofies though choking on a wall i might go to oz take some niggas on


blue yelling fuck the pharaohs and pharisees moses is back i wonder if gladys smiles when angels bring my butt to church but i made it you know that you could not mess


guns to jesus come back back back and forth with fifths of jack and fourths of weed im back up back up off me back up off me back up in this bitch


drugs i know its gonna rain mama you gon torture me well thats awfully fair all for nothing tryna tweak me like you to have to have a pot of gold too rich


brother foot on the road again so bitch lets fuck so i can do is laugh i say dont you end up on world star getting your ass on old ass on my


cash money i need cash money so back up off me back up in this house cause its so crisp the most brokest cold stock broker winter solstice i could hurry but i


apple trees will be changed when it rains will they shout to you from